### Data Preprocessing

In [3]:
# Data Preprocessing
import numpy as np
import pandas as pd

# Read sales_sum data
sales_sum_data = pd.read_csv('t_sales_sum.csv',header=0,index_col=0)
sales_sum_data = sales_sum_data.drop_duplicates()
n_sales_sum_features = len(sales_sum_data.columns) - 1
n_sales_sum = len(sales_sum_data.index)

print "sales_sum data read successfully!"
print "Total number of sales_sums: {}".format(n_sales_sum)
print "Number of features: {}".format(n_sales_sum_features)

sales_sum data read successfully!
Total number of sales_sums: 24000
Number of features: 1


In [ ]:
import sklearn
version = sklearn.__version__
print(version)

In [ ]:
# Read training data form csv 
ads_train_data = pd.read_csv('ads_train_data.csv')
comment_train_data = pd.read_csv('comment_train_data.csv')
order_train_data = pd.read_csv('order_train_data.csv')
product_train_data = pd.read_csv('product_train_data.csv')
sales_sum_train_data = pd.read_csv('sales_sum_train_data.csv')
sales_sum_train_data = sales_sum_train_data.drop_duplicates() # drop duplicate data

# Read predict data
ads_target_data = pd.read_csv('ads_target_data.csv')
comment_target_data = pd.read_csv('comment_target_data.csv')
order_target_data = pd.read_csv('order_target_data.csv')
product_target_data = pd.read_csv('product_target_data.csv')

In [ ]:
# merge the training data
train_data = pd.merge(sales_sum_train_data,product_train_data,how ='left', on=['shop_id','dt'])
train_data = pd.merge(train_data,comment_train_data,how ='left', on=['shop_id','dt'])
train_data = pd.merge(train_data,order_train_data,how ='left', on=['shop_id','dt'])
train_data = pd.merge(train_data,ads_train_data,how ='left', on=['shop_id','dt'])

# calculate the mean
product_on_mean = format(train_data['product_on'].mean(),'.2f')
sum_dis_num_mean  = format(train_data['sum_dis_num'].mean(),'.2f')
bad_percent_mean  = format(train_data['bad_percent'].mean(),'.2f')
good_percent_mean = format(train_data['good_percent'].mean(),'.2f')
sum_sale_amt_mean   = format(train_data['sum_sale_amt'].mean(),'.2f')
sum_rtn_amt_mean   = format(train_data['sum_rtn_amt'].mean(),'.2f')
sum_offer_amt_mean  = format(train_data['sum_offer_amt'].mean(),'.2f') 
sum_ord_cnt_mean  = format(train_data['sum_ord_cnt'].mean(),'.2f')
sum_user_cnt_mean   = format(train_data['sum_user_cnt'].mean(),'.2f') 
sum_charge_mean   = format(train_data['sum_charge'].mean(),'.2f')
sum_consume_mean = format(train_data['sum_consume'].mean(),'.2f')
# calculate the median
sum_charge_median   = format(train_data['sum_charge'].median(),'.2f')
sum_consume_median = format(train_data['sum_consume'].median(),'.2f')

# Handle missing values
# Most use the mean, a small part of the median
train_data = train_data.fillna({'product_on':product_on_mean,
                                'sum_dis_num':sum_dis_num_mean,
                                'bad_percent':bad_percent_mean,
                                'good_percent':good_percent_mean,
                                'sum_sale_amt':sum_sale_amt_mean,
                                'sum_rtn_amt':sum_rtn_amt_mean,
                                'sum_offer_amt':sum_offer_amt_mean,
                                'sum_ord_cnt':sum_ord_cnt_mean,
                                'sum_user_cnt':sum_user_cnt_mean,
                                'sum_charge':sum_charge_median,
                                'sum_consume':sum_consume_median
                               }) 

#print(test_data[test_data.isnull().values==True]) # check NaN
#print(train_data)

# Output training data
columns = ['shop_id','dt','sale_amt_3m','product_on','sum_dis_num','bad_percent','good_percent',
           'sum_sale_amt','sum_rtn_amt','sum_ord_cnt','sum_user_cnt','sum_offer_amt','sum_charge','sum_consume']

train_data.to_csv('train_data.csv',index = False, header = True, columns = columns) 

In [ ]:
# Merge target forecast data 

target_data = pd.merge(order_target_data,ads_target_data, how ='left', on=['shop_id','dt'])
target_data = pd.merge(target_data,comment_target_data, how ='left', on=['shop_id','dt'])
target_data = pd.merge(target_data,product_target_data, how ='left', on=['shop_id','dt'])

# calculate the mean and median
target_product_on_mean = format(target_data['product_on'].mean(),'.2f')
target_sum_dis_num_mean  = format(target_data['sum_dis_num'].mean(),'.2f')
target_bad_percent_mean  = format(target_data['bad_percent'].mean(),'.2f')
target_good_percent_mean = format(target_data['good_percent'].mean(),'.2f')
target_sum_charge_median   = format(target_data['sum_charge'].median(),'.2f')
target_sum_consume_median = format(target_data['sum_consume'].median(),'.2f')

# Handle missing values
# Most use the mean, a small part of the median
target_data = target_data.fillna({  'product_on':target_product_on_mean,# use mean
                                'sum_dis_num':target_sum_dis_num_mean,
                                'bad_percent':target_bad_percent_mean,
                                'good_percent':target_good_percent_mean,
                                'sum_charge':target_sum_charge_median,# use median
                                'sum_consume':target_sum_consume_median
                               }) 

# print(target_data[target_data.isnull().values==True]) # check NaN data 
# print(target_data)

# Output data use to predict 
columns = ['shop_id','dt','product_on','sum_dis_num','bad_percent','good_percent',
           'sum_sale_amt','sum_rtn_amt','sum_ord_cnt','sum_user_cnt','sum_offer_amt','sum_charge','sum_consume']
target_data.to_csv('target_data.csv',index = False, header = True, columns = columns) 

In [ ]:
# Visualization 
import matplotlib.pyplot as plt

groups = sales_sum_data.groupby(sales_sum_data['shop_id'])
# n_groups = len(groups)
for name,group in groups:
    #print(group.sort_index())
    group.sort_index().plot(y='sale_amt_3m')
#groups.plot(y='sale_amt_3m')    
plt.show()

In [ ]:
# Beanchmark
# Mean of sale_amt_3m for each store for 8 months
grouped = sales_sum_data['sale_amt_3m'].groupby(sales_sum_data['shop_id'])
print(grouped.mean()) 

### Implementation

In [ ]:
# Implementation

train = pd.read_csv('train_data.csv')
target = pd.read_csv('target_data.csv')
#selected_features=['consume','good','bad','sale','product','dis','cnmt','ord_cnt','user_cnt','offer_amt']
selected_features=['product_on','sum_dis_num','bad_percent','good_percent',
                   'sum_sale_amt','sum_rtn_amt','sum_ord_cnt','sum_user_cnt',
                   'sum_offer_amt','sum_charge','sum_consume']
features = train[selected_features]
X_target = target[selected_features]

sale_amt_3m = train['sale_amt_3m']

print "The dataset has {} data points with {} variables each.".format(*features.shape)
print "The predict dataset has {} data points with {} variables each.".format(*X_target.shape)


In [ ]:
# From sklearn.cross_validation import data splitter
from sklearn.model_selection import train_test_split

X = features
y = sale_amt_3m

# A random sample of 25% of the data to build test samples, the rest as a training sample
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=33)

# Success
print "Training and testing split was successful."


In [ ]:
# TODO: Minimum sale_amt_3m of the data
minimum_sale_amt_3m = np.min(sale_amt_3m)

# TODO: Maximum sale_amt_3m of the data
maximum_sale_amt_3m = np.max(sale_amt_3m)

# TODO: Mean sale_amt_3m of the data
mean_sale_amt_3m = np.mean(sale_amt_3m)

# TODO: Median sale_amt_3m of the data
median_sale_amt_3m = np.median(sale_amt_3m)

# TODO: Standard deviation of sale_amt_3m of the data
std_sale_amt_3m = np.std(sale_amt_3m)

# Show the calculated statistics
print "Statistics for StoreForecast dataset:\n"
print "Minimum sale_amt_3m: ${:,.2f}".format(minimum_sale_amt_3m)
print "Maximum sale_amt_3m: ${:,.2f}".format(maximum_sale_amt_3m)
print "Mean sale_amt_3m: ${:,.2f}".format(mean_sale_amt_3m)
print "Median sale_amt_3m ${:,.2f}".format(median_sale_amt_3m)
print "Standard deviation of sale_amt_3m: ${:,.2f}".format(std_sale_amt_3m)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
ss_X = StandardScaler()
ss_y = StandardScaler()

# Standard Scaler
X_train = ss_X.fit_transform(X_train)

X_test = ss_X.fit_transform(X_test)
y_train = ss_y.fit_transform(y_train.reshape(-1,1))
y_test = ss_y.fit_transform(y_test.reshape(-1,1))
X_target = ss_X.fit_transform(X_target)

In [ ]:
# Use Linear Regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_y_predict = lr.predict(X_test)

#lr.fit(X_train_ss,y_train_ss)
#lr_y_predict_ss = lr.predict(X_test_ss)
# print(lr_y_predict)
# print(ss_y.inverse_transform(lr_y_predict))

In [ ]:
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error

print 'The value of R-squared of LinearRegression is',r2_score(y_test,lr_y_predict)
#print 'The mean squared error of LinearRegression is',mean_squared_error(y_test,lr_y_predict)
#print 'The mean absoluate error of LinearRegression is',mean_absolute_error(y_test,lr_y_predict)
#print 'The value of R-squared of LinearRegression is',r2_score(y_test_ss,lr_y_predict_ss)
print 'The mean squared error of LinearRegression is',mean_squared_error(ss_y.inverse_transform(y_test),ss_y.inverse_transform(lr_y_predict))
print 'The mean absoluate error of LinearRegression is',mean_absolute_error(ss_y.inverse_transform(y_test),ss_y.inverse_transform(lr_y_predict))

In [ ]:
from sklearn.svm import SVR

# use linear kernel 
linear_svr = SVR(kernel='linear')
linear_svr.fit(X_train, y_train)
linear_svr_y_predict = linear_svr.predict(X_test)

# use ploy kernel
#poly_svr = SVR(kernel='ploy')
#poly_svr.fit(X_train, y_train)
#ploy_svr_y_predit = poly_svr.predict(X_test)

# use rbf kernel
rbf_svr = SVR(kernel='rbf')
rbf_svr.fit(X_train, y_train)
rbf_svr_y_predit = rbf_svr.predict(X_test)



In [ ]:
print 'The value of R-squared of Linear SVR is',linear_svr.score(X_test,y_test)
print 'The mean squared error of Linear SVR is',mean_squared_error(ss_y.inverse_transform(y_test),
                                                                         ss_y.inverse_transform(linear_svr_y_predict))
print 'The mean absoluate error of Linear SVR is',mean_absolute_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(linear_svr_y_predict))
#print 'The value of R-squared of Linear SVR is',poly_svr.score(X_test,y_test)
#print 'The mean squared error of Linear SVR is',mean_squared_error(ss_y.inverse_transform(y_test),ss_y.inverse_transform(ploy_svr_y_predit))
#print 'The mean absoluate error of Linear SVR is',mean_absolute_error(ss_y.inverse_transform(y_test),ss_y.inverse_transform(ploy_svr_y_predit))
print 'The value of R-squared of RBF SVR is',rbf_svr.score(X_test,y_test)
print 'The mean squared error of RBF SVR is',mean_squared_error(ss_y.inverse_transform(y_test),
                                                                         ss_y.inverse_transform(rbf_svr_y_predit))
print 'The mean absoluate error of RBF SVR is',mean_absolute_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(rbf_svr_y_predit))


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# init KNeighborsRegressor,use weights='uniform'
uni_knr = KNeighborsRegressor(weights='uniform')
uni_knr.fit(X_train,y_train)
uni_knr_y_predict =uni_knr.predict(X_test)

# init KNeighborsRegressor,weight points by the inverse of their distance weights='distance'
dis_knr = KNeighborsRegressor(weights='distance')
dis_knr.fit(X_train,y_train)
dis_knr_y_predict =dis_knr.predict(X_test)

print 'The value of R-squared of uniform-weights KNeighborsRegressor is',uni_knr.score(X_test,y_test)
print 'The mean squared error of uniform-weights KNeighborsRegressor is',mean_squared_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(uni_knr_y_predict))
print 'The mean absoluate error of uniform-weights KNeighborsRegressor is',mean_absolute_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(uni_knr_y_predict))

print 'The value of R-squared of distance-weights KNeighborsRegressor is',dis_knr.score(X_test,y_test)
print 'The mean squared error of distance-weights KNeighborsRegressor is',mean_squared_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(dis_knr_y_predict))
print 'The mean absoluate error of distance-weights KNeighborsRegressor is',mean_absolute_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(dis_knr_y_predict))

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
dtr_y_predict = dtr.predict(X_test)

print 'The value of R-squared of DecisionTreeRegressor is',dtr.score(X_test,y_test)
print 'The mean squared error of DecisionTreeRegressor is',mean_squared_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(dtr_y_predict))
print 'The mean absoluate error of DecisionTreeRegressor is',mean_absolute_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(dtr_y_predict))

In [ ]:
# Use enemble
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor

# use RandomForestRegressor
rtf = RandomForestRegressor()
rtf.fit(X_train,y_train)
rtf_y_predict = rtf.predict(X_test)

# use ExtraTreesRegressor
etr = ExtraTreesRegressor()
etr.fit(X_train,y_train)
etr_y_predict = etr.predict(X_test)

# use GradientBoostingRegressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train,y_train)
gbr_y_predict = gbr.predict(X_test)

print 'The value of R-squared of RandomForestRegressor is',rtf.score(X_test,y_test)
print 'The mean squared error of RandomForestRegressor is',mean_squared_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(rtf_y_predict))
print 'The mean absoluate error of RandomForestRegressor is',mean_absolute_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(rtf_y_predict))

print 'The value of R-squared of ExtraTreesRegressor is',etr.score(X_test,y_test)
print 'The mean squared error of ExtraTreesRegressor is',mean_squared_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(etr_y_predict))
print 'The mean absoluate error of ExtraTreesRegressor is',mean_absolute_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(etr_y_predict))

print 'The value of R-squared of GradientBoostingRegressor is',gbr.score(X_test,y_test)
print 'The mean squared error of GradientBoostingRegressor is',mean_squared_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(gbr_y_predict))
print 'The mean absoluate error of GradientBoostingRegressor is',mean_absolute_error(ss_y.inverse_transform(y_test),
                                                                          ss_y.inverse_transform(gbr_y_predict))

#### The MAE score of the SVR with RBF kernel is the highest and this algorithm is chosen

In [ ]:
# Output to csv file
y_test_predict = rbf_svr.predict(X_target)
y_test_predict = ss_y.inverse_transform(y_test_predict)
jd_best_submission =pd.DataFrame({'shop_id':target['shop_id'],'sale_amt_3m':y_test_predict,})

# set columns 
columns=['shop_id','sale_amt_3m']

# output csv， index，header，columns
jd_best_submission.to_csv('Sales_Forecast_Upload_3.csv',index=False, header = False, columns=columns)

### Refinement

In [ ]:
# TODO: Import 'mean_absolute_error'
from sklearn.metrics import mean_absolute_error
def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = mean_absolute_error(y_true, y_predict)
    
    # Return the score
    return score

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import make_scorer

def svr_param_selection(X, y):
    # Create cross-validation sets from the training data
    # sklearn version 0.18: ShuffleSplit(n_splits=10, test_size=0.1, train_size=None, random_state=None)
    # sklearn versiin 0.17: ShuffleSplit(n, n_iter=10, test_size=0.1, train_size=None, random_state=None)
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)
    # cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    # TODO: Create a SVR  object
    regressor = SVR(kernel='rbf')
    
    # Create a dictionary for the parameter Cs,gammas
    Cs = [0.001,0.01,0.1,1,10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma': gammas}
    
    #Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)
    
    grid_search = GridSearchCV(regressor, param_grid, scoring = scoring_fnc, cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid_search.fit(X,y)
    
    # Return the optimal model after fitting the data
    return grid_search.best_estimator_

In [ ]:
# Fit the training data to the model using grid search
svr_opt = svr_param_selection(X_train, y_train)
print("Parameter 'C' is {} for the optimal model".format(svr_opt.get_params()['C']))
print("Parameter 'gamma' is {} for the optimal model".format(svr_opt.get_params()['gamma']))

In [ ]:
# Output to csv file
y_predict_opt = svr_opt.predict(X_target)
y_predict_opt = ss_y.inverse_transform(y_predict_opt)
jd_best_submission_opt =pd.DataFrame({'shop_id':target['shop_id'],'sale_amt_3m':y_predict_opt,})

# set columns 
columns=['shop_id','sale_amt_3m']

# output csv， index，header，columns
jd_best_submission_opt.to_csv('Sales_Forecast_Upload_3_opt.csv',index=False, header = False, columns=columns)
print "Create csv file successful."

In [8]:
vs_data = pd.read_csv('vs.csv',header=0,index_col=0)
vs_data = vs_data.drop_duplicates()
n_vs_features = len(vs_data.columns) - 1
n_vs = len(vs_data.index)

print "sales_sum data read successfully!"
print "Total number of sales_sums: {}".format(n_vs_features)
print "Number of features: {}".format(n_vs)

sales_sum data read successfully!
Total number of sales_sums: 1
Number of features: 27000


In [9]:
vss = vs_data.groupby(vs_data['shop_id'])
# n_groups = len(groups)
for name,vs in vss:
    #print(group.sort_index())
    vs.sort_index().plot(y='sale_amt_3m')
#groups.plot(y='sale_amt_3m')    
plt.show()

/Users/saneryee/anaconda2/lib/python2.7/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


NameError: name 'plt' is not defined